# Pydantic Basics

## What You'll Learn
- What Pydantic is and why it's useful
- Creating models with BaseModel
- Automatic data validation and type coercion
- Converting models to dictionaries and JSON
- Common validation errors and how to handle them
- Differences between dataclasses and Pydantic models

---

## What is Pydantic?

**Pydantic** is a Python library for data validation and settings management using type hints. It's like dataclasses, but with automatic validation and parsing.

### Why Use Pydantic?

Dataclasses help with structure, but they don't validate data:

In [ ]:
from dataclasses import dataclass

# ❌ Dataclasses don't validate types
@dataclass
class User:
    username: str
    age: int

# This works but shouldn't - age should be int!
user = User("alice", "twenty-five")  # No error!
print(user)
print(f"Age type: {type(user.age)}")

**Problems with dataclasses:**
- No automatic type validation
- No data parsing (converting string "25" to int 25)
- No field-level validation rules
- Can create invalid data structures

---

## Installing Pydantic

First, install Pydantic (version 2.x):

```bash
pip install pydantic
```

---

## Creating Pydantic Models

### Basic Model

Use `BaseModel` instead of `@dataclass`:

In [ ]:
from pydantic import BaseModel

# ✅ Pydantic model with validation
class User(BaseModel):
    """User model with automatic validation."""
    username: str
    age: int

# Valid user
user1 = User(username="alice", age=25)
print(user1)
print(f"Age type: {type(user1.age)}")

In [ ]:
# Try creating user with wrong type
try:
    user2 = User(username="bob", age="twenty-five")
except Exception as e:
    print(f"Error: {e}")

**What happens:**
1. Pydantic validates that `age` is an integer
2. If validation fails, raises `ValidationError` with details
3. Unlike dataclasses, **validation happens automatically** on creation

---

## Automatic Type Coercion

Pydantic can **convert** compatible types automatically:

In [ ]:
from pydantic import BaseModel

class Product(BaseModel):
    """Product with price."""
    name: str
    price: float
    quantity: int

# Pydantic converts string numbers to correct types
product = Product(name="Laptop", price="999.99", quantity="5")

print(product)
print(f"Price type: {type(product.price)}")
print(f"Quantity type: {type(product.quantity)}")

**What happens:**
1. `price="999.99"` (string) → converted to `999.99` (float)
2. `quantity="5"` (string) → converted to `5` (int)
3. This is called **type coercion** - automatic conversion of compatible types
4. Very useful when parsing JSON, form data, or API responses

---

## Default Values and Optional Fields

In [ ]:
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    """User with optional fields."""
    username: str
    email: str
    age: int = 18  # Default value
    bio: Optional[str] = None  # Optional field
    is_active: bool = True

# Create users with different combinations
user1 = User(username="alice", email="alice@example.com")
user2 = User(username="bob", email="bob@example.com", age=30, bio="Python dev")

print(user1)
print(user2)

---

## Model Methods: dict() and model_dump_json()

Pydantic models can easily convert to dictionaries and JSON:

In [ ]:
from pydantic import BaseModel

class Product(BaseModel):
    """Product model."""
    name: str
    price: float
    in_stock: bool

product = Product(name="Laptop", price=999.99, in_stock=True)

# Convert to dictionary
product_dict = product.model_dump()
print(f"Dictionary: {product_dict}")
print(f"Type: {type(product_dict)}")

# Convert to JSON string
product_json = product.model_dump_json()
print(f"\nJSON: {product_json}")
print(f"Type: {type(product_json)}")

### Parsing from Dictionary or JSON

In [ ]:
import json
from pydantic import BaseModel

class User(BaseModel):
    """User model."""
    username: str
    age: int

# From dictionary
user_data = {"username": "alice", "age": 25}
user1 = User(**user_data)
print(f"From dict: {user1}")

# From JSON string
json_string = '{"username": "bob", "age": 30}'
user2 = User.model_validate_json(json_string)
print(f"From JSON: {user2}")

---

## Validation Errors

When validation fails, Pydantic provides detailed error messages:

In [ ]:
from pydantic import BaseModel, ValidationError

class Product(BaseModel):
    """Product model."""
    name: str
    price: float
    quantity: int

# Try creating product with invalid data
try:
    product = Product(
        name="Laptop",
        price="not-a-number",  # Invalid
        quantity="also-invalid"  # Invalid
    )
except ValidationError as e:
    print("Validation failed!")
    print(e)

### Accessing Error Details

In [ ]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    """User model."""
    username: str
    age: int

try:
    user = User(username=123, age="invalid")
except ValidationError as e:
    # Get errors as list of dictionaries
    errors = e.errors()
    
    print("Individual errors:")
    for error in errors:
        print(f"- Field: {error['loc'][0]}")
        print(f"  Error: {error['msg']}")
        print(f"  Type: {error['type']}\n")

---

## Comparison: Dataclass vs Pydantic Model

| Feature | Dataclass | Pydantic Model |
|---------|-----------|----------------|
| Type validation | ❌ No | ✅ Yes |
| Type coercion | ❌ No | ✅ Yes |
| Field validation | ❌ No | ✅ Yes |
| JSON parsing | Manual | ✅ Built-in |
| Performance | Faster | Slightly slower |
| Setup | Built-in | Requires install |
| Best for | Simple data storage | APIs, configs, validation |

### Side-by-Side Example

In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel

# Dataclass version
@dataclass
class UserDataclass:
    username: str
    age: int

# Pydantic version
class UserPydantic(BaseModel):
    username: str
    age: int

# Both accept correct types
dc_user = UserDataclass("alice", 25)
pyd_user = UserPydantic(username="alice", age=25)

print(f"Dataclass: {dc_user}")
print(f"Pydantic: {pyd_user}")

# Dataclass: No validation (accepts wrong types)
dc_wrong = UserDataclass("bob", "thirty")  # Works but wrong!
print(f"\nDataclass with wrong type: {dc_wrong}")

# Pydantic: Validates (raises error)
try:
    pyd_wrong = UserPydantic(username="bob", age="thirty")
except Exception as e:
    print(f"\nPydantic error: {e}")

---

## Practical Example: API User Model

In [ ]:
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

class UserProfile(BaseModel):
    """User profile from API."""
    user_id: int
    username: str
    email: str
    full_name: Optional[str] = None
    is_active: bool = True
    created_at: Optional[datetime] = None

# Simulate API response (JSON string)
api_response = '''
{
    "user_id": "123",
    "username": "alice",
    "email": "alice@example.com",
    "full_name": "Alice Smith",
    "is_active": "true",
    "created_at": "2024-01-15T10:30:00"
}
'''

# Parse and validate
user = UserProfile.model_validate_json(api_response)

print(user)
print(f"\nUser ID type: {type(user.user_id)}")
print(f"Is active type: {type(user.is_active)}")
print(f"Created at type: {type(user.created_at)}")

**What happens:**
1. JSON has strings: `"123"`, `"true"`, `"2024-01-15T10:30:00"`
2. Pydantic automatically converts them to correct types: `int`, `bool`, `datetime`
3. Data is validated - if email is invalid format, validation fails
4. Perfect for parsing API responses, configuration files, form data

---

## When to Use Pydantic

### ✅ Use Pydantic When:
- Parsing external data (APIs, JSON, forms)
- Need automatic data validation
- Need type coercion (string to int, etc.)
- Building APIs (especially with FastAPI)
- Loading configuration files
- Working with user input

### ✅ Use Dataclasses When:
- Simple internal data structures
- No validation needed
- Performance is critical
- Don't want external dependencies

---

## Best Practices

### ✅ Do:
- Use Pydantic for external data (APIs, configs, user input)
- Add docstrings to models
- Use `Optional[Type]` for fields that can be None
- Handle `ValidationError` when parsing untrusted data
- Use `model_dump()` to convert to dict, `model_dump_json()` to JSON

### ❌ Don't:
- Don't use Pydantic if simple dataclasses suffice
- Don't ignore validation errors (catch and handle them)
- Don't use Pydantic for internal data that never needs validation
- Don't forget to install Pydantic (`pip install pydantic`)

### Examples:

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import Optional

# ✅ Good: Model for API response
class APIResponse(BaseModel):
    """Standard API response."""
    status: str
    data: Optional[dict] = None
    error: Optional[str] = None

# ✅ Good: Handle validation errors
def parse_user_input(data: dict):
    """Parse user input safely."""
    try:
        return APIResponse(**data)
    except ValidationError as e:
        print(f"Invalid input: {e}")
        return None

# Test
valid_data = {"status": "success", "data": {"id": 1}}
result = parse_user_input(valid_data)
print(f"Result: {result}")

---

## Summary

### Key Concepts:
- **Pydantic** provides data validation using type hints
- Inherit from `BaseModel` instead of using `@dataclass`
- **Automatic validation** on instance creation
- **Type coercion** converts compatible types automatically
- `model_dump()` converts to dict, `model_dump_json()` to JSON
- `model_validate_json()` parses JSON strings
- `ValidationError` provides detailed error information
- Use for APIs, configs, and external data parsing

### Syntax Reference:
```python
from pydantic import BaseModel, ValidationError
from typing import Optional

# Define model
class User(BaseModel):
    username: str
    age: int
    bio: Optional[str] = None

# Create instance
user = User(username="alice", age=25)

# Convert to dict/JSON
user_dict = user.model_dump()
user_json = user.model_dump_json()

# Parse from JSON
user2 = User.model_validate_json('{"username":"bob","age":30}')

# Handle errors
try:
    invalid = User(username="charlie", age="invalid")
except ValidationError as e:
    print(e)
```

### Next Steps:
Next, learn about [Pydantic Validation](04-pydantic-validation.ipynb) to add custom validation rules and use advanced validators.